<a href="https://colab.research.google.com/github/akshatchaturvedi28/Collab-TensorFlow/blob/master/Regression_BostonHousing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd

tf.random.set_seed(7)

     |████████████████████████████████| 86.3MB 47kB/s 
     |████████████████████████████████| 4.3MB 46.0MB/s 
     |████████████████████████████████| 501kB 51.5MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
from tensorflow import keras

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [0]:
train, test = train_test_split(data, test_size=0.25, random_state=7)
train, val = train_test_split(train, test_size=0.25, random_state=7)
print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

284 train examples
95 validation examples
127 test examples


Converting the Pandas DataFrames into Tensorflow Datasets

In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, True, batch_size)
val_ds = df_to_dataset(val, False, batch_size)
test_ds = df_to_dataset(test, False, batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


**Defining Feature Columns**

In [0]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [0]:
# define feature_columns as a list of features using functions from tf.feature_column


feature_columns = [tf.feature_column.numeric_column('CRIM'),
tf.feature_column.numeric_column('ZN'),
tf.feature_column.numeric_column('INDUS'),
tf.feature_column.numeric_column('CHAS'),
tf.feature_column.numeric_column('NOX'),
tf.feature_column.numeric_column('RM'),
tf.feature_column.numeric_column('AGE'),
tf.feature_column.numeric_column('DIS'),
tf.feature_column.bucketized_column(tf.feature_column.numeric_column('RAD'), boundaries=[2, 5]),
tf.feature_column.numeric_column('TAX'),
tf.feature_column.numeric_column('PTRATIO'),
tf.feature_column.numeric_column('B'),
tf.feature_column.numeric_column('LSTAT')]
feature_columns

[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 BucketizedColumn(source_column=NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(2, 5)),
 NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32

**Building the model**

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
model = keras.Sequential([
    feature_layer,
    keras.layers.Dense(1, activation=None)
])

In [0]:
model.compile(optimizer='adam', loss=tf.keras.losses.MAPE , metrics=['mse'])

In [0]:
model.fit(train_ds, validation_data=val_ds, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 9ms/step - loss: 21.3807 - mse: 52.5584 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00
Epoch 2/200
9/9 [==============================] - 0s 5ms/step - loss: 21.4997 - mse: 50.5495 - val_loss: 21.4901 - val_mse: 48.3737
Epoch 3/200
9/9 [==============================] - 0s 5ms/step - loss: 21.4298 - mse: 52.7247 - val_loss: 21.7170 - val_mse: 50.1429
Epoch 4/200
9/9 [==============================] - 0s 5ms/step - loss: 21.4174 - mse: 51.6521 - val_loss: 21.3340 - val_mse: 48.3871
Epoch 5/200
9/9 [==============================] - 0s 6ms/step - loss: 21.2133 - mse: 52.1970 - val_loss: 21.6569 - val_mse: 50.8281
Epoch 6/200
9/9 [==============================] - 0s 5ms/step - loss: 21.3735 - mse: 53.2310 - val_loss: 21.4587 - val_mse: 49.4973
Epoch 7/200
9/9 [==============================] - 0s 6ms/step - loss: 21.5435 - mse: 52.4324 - val_loss: 21.3742 - val_mse: 48.8467
Epoch 8/200
9/9 [==============================] - 0s 5ms/step 

In [0]:
loss, mse = model.evaluate(test_ds)
print("Mean Squared Error - Test Data", mse)

4/4 [==============================] - 0s 3ms/step - loss: 18.7094 - mse: 47.9443
Mean Squared Error - Test Data 47.944283
